In [ ]:
# !pip install pyarrow

In [ ]:
from datetime import datetime

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import pickle

#import xgbtune 

In [ ]:
df = pd.read_parquet("../data/dataset.parquet")
df

In [ ]:
# target = "target_close"
target = "close_percent_change_1"

x = df.drop(columns=[target]).set_index("date")
y = df[["date", target]].set_index("date")
y

In [ ]:
x

In [ ]:
table_info = pd.DataFrame()

# Print columns that are not int, float, bool or category
for col in x.columns:
    dtype = x[col].dtype
    if dtype not in ["int", "float64", "bool", "category"]:
        table_info = pd.concat([table_info, pd.DataFrame({"Column Name": [col], "Data Type": [x[col].dtype]})])

table_info

In [ ]:
# Convert columns to float if they are not int, float, bool or category. Handle Cannot cast DatetimeArray to dtype float64 (XGBoosted models cannot use strings, but categories as enumerated values)
for col in x.columns:
    dtype = x[col].dtype
    if dtype not in ["int", "float64", "bool", "category"]:
        try:
            x[col] = x[col].astype("float")
        except:
            # drop datetime columns
            x = x.drop(columns=[col])

            print(f"Dropped Column: {col}")

In [ ]:
# # Normalizing the features between 0 and 1
# y_scaler = MinMaxScaler()
# y = y_scaler.fit_transform(y.values.reshape(-1, 1)).flatten()

# x_scaler = MinMaxScaler()
# x = x_scaler.fit_transform(x)

# y

In [ ]:
#    All Features   | Target
# +-----------------+---------+
# | x_train         | y_train | <- 85% of the data which is used for training
# |                 |         |
# +-----------------+---------+
# | x_test          | y_test  | <- 15% of the data which is used for testing
# +-----------------+---------+


# x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.15, shuffle=False)
# x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.5, shuffle=True)
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, shuffle=True)
# x_train

In [ ]:
# y_test

In [ ]:
x_train

In [ ]:
import numpy

from itertools import count, takewhile
def frange(start, stop, step):
    return takewhile(lambda x: x< stop, count(start, step))

# list(frange(0, 1, 0.1))
numpy.linspace(5, 10, num=6).tolist()

In [ ]:
# model = xgb.XGBRegressor(n_estimators=100, max_depth=7, eta=0.1, subsample=1, colsample_bytree=.3)
# model = xgb.XGBRegressor(n_estimators=150, max_depth=7, eta=0.05, subsample=1, colsample_bytree=.3)
# model = xgb.XGBRegressor(n_estimators=100, max_depth=7, eta=0.05, subsample=1, colsampvle_bytree=.3)
model = xgb.XGBRegressor(n_estimators=500, max_depth=7, eta=0.05, subsample=0.9, colsample_bytree=0.4)
model.fit(x_train, y_train)

from sklearn.experimental import enable_halving_search_cv
from sklearn.model_selection import GridSearchCV, HalvingGridSearchCV, HalvingRandomSearchCV

tune_params = {
    # "n_estimators": [100, 150],
    # "max_depth": [5, 6,],
    # "learning_rate": [0.05, 0.1],

    "n_estimators": [400, 450, 500, 550, 600],
    "max_depth": [6, 7, 8, 9, 11],
    "learning_rate": [0.05, 0.1, 0.2, 0.3, 0.4,],
    "subsample": [0.7, 0.8, 0.9, 1.0, 1.1],
    "colsample_bytree": [0.1, 0.2, 0.3, 0.4, 0.5],

}

# {'subsample': 0.9, 'n_estimators': 500, 'max_depth': 7, 'learning_rate': 0.05, 'colsample_bytree': 0.3}
# {'subsample': 0.8, 'n_estimators': 400, 'max_depth': 7, 'learning_rate': 0.05, 'colsample_bytree': 0.2}

# xgb_model = xgb.XGBRegressor()
# # model = GridSearchCV(estimator=xgb_model, param_grid=tune_params)
# model = HalvingRandomSearchCV(estimator=xgb_model, param_distributions=tune_params)
# model.fit(x_train, y_train)

# print(model.best_params_)

pickle.dump(model, open("../model/xgboost_model.pkl", "wb")) # Save model as Python pickle object
model.save_model("../model/xgboost_model.json")

In [ ]:
# print(model.best_params_)
# model


In [ ]:
predicted = model.predict(x_test)
y_test["predicted"] = predicted
y_test

In [ ]:
# Get the root mean squared error (RMSE)
mse = np.mean((y_test["predicted"] - y_test[target]) ** 2)
rmse = np.sqrt(mse)
rmse

In [ ]:
# Get the mean absolute error (MAE)
mae = np.mean(np.abs(y_test["predicted"] - y_test[target]))
mae

In [ ]:
# 0.01455269590461311
# 0.013673832772506729
# 0.012685448862479572

In [ ]:
# rmsle
rmsle = np.sqrt(np.mean(np.log1p(np.abs(y_test["predicted"] - y_test[target])) ** 2))
rmsle

In [ ]:
# Get the mean absolute percentage error (MAPE)
mape = np.mean(np.abs((y_test[target] - y_test["predicted"]) / y_test[target])) * 100
mape

In [ ]:
# Save predicted values
y_test.sort_index().to_parquet("../data/predicted.parquet")